<a href="https://colab.research.google.com/github/matanlakker/BINA/blob/main/Transfer_Learning_Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/"

'ana film (1).mp4'	      'Hoveret (1).gdoc'
'ana film (2).mp4'	      'Hoveret (2).gdoc'
'ana film (3).mp4'	       Hoveret.gdoc
'ana film.mp4'		       IMG_0308.MOV
'Apr 20, 2020 11:17:22.jpeg'   Screenshot_20190310-101212_WhatsApp.jpg
'Apr 20, 2020 11:19:59.jpeg'  'Untitled form (1).gform'
'Apr 20, 2020 11:20:22.jpeg'  'Untitled form (2).gform'
'Apr 24, 2020 11:20:00.jpeg'  'Untitled form (3).gform'
'Apr 24, 2020 11:20:26.jpeg'  'Untitled form.gform'
'Apr 24, 2020 11:20:45.jpeg'   work1.html
'Apr 24, 2020 11:21:04.jpeg'  '‎⁨אפוקסידים שיעור 2024-2025⁩.pdf'
'Apr 24, 2020 11:21:21.jpeg'  "בת מצוה- עילית שובו אשדוד ע''ד.mp4"
 chaya.gaya.mp4		      'הצעות_לנושאים_לעבודת_חקר_על_ספר_בראשית (4)1 (2).doc'
 Classroom		      'כתיבה אקדמית.docx'
'Colab Notebooks'	      'מחוון לעבודות תמונות מספרות (1).docx'
'Document (1).docx'	      'מטלה 2 מתוקן (1).docx'
 Document.docx		      'מטלה 2 מתוקן.docx'
'English project.docx'	      'מטלה_להגשה_בהיסטוריה[1].gdoc'
'English project.gdoc'	      'סיכומים מתמט

In [ ]:
!ls "/content/drive/MyDrive/BINA PROJECT/indian Worker/indianWorker"


ls: cannot access '/content/drive/MyDrive/BINA PROJECT/indian Worker/indianWorker': No such file or directory


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import matplotlib.pyplot as plt
import random
import shutil
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# --- הגדרות בסיסיות ---
batch_size = 32
img_size = (224, 224)  # גודל התמונה המתאים למודל ResNet50

# --- חלוקת התמונות ל-Train, Validation ו-Test ---
source_dir_light = "/content/drive/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/light"  # נתיב לתמונות קלות
source_dir_severe = "/content/drive/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/severe"  # נתיב לתמונות חמורות
output_dir = "/content/drive/MyDrive/BINA PROJECT/indian Worker/indianWorker/split_data_resnet50"  # נתיב לשמירת התמונות המחולקות

# אחוזי חלוקה
test_split = 0.1 #  10% לקבוצת הבדיקה
val_split = 0.3  # 30% לקבוצת הולידציה

In [ ]:

# פונקציה לחלוקת תמונות
def split_data(source_dir, output_dir, category, test_split, val_split):
    files = os.listdir(source_dir)
    random.shuffle(files)

    test_size = int(len(files) * test_split)
    val_size = int(len(files) * val_split)

    test_files = files[:test_size]
    val_files = files[test_size:test_size + val_size]
    train_files = files[test_size + val_size:]

    # יצירת תיקיות
    for split, split_files in zip(['train', 'val', 'test'], [train_files, val_files, test_files]):
        split_dir = os.path.join(output_dir, split, category)
        os.makedirs(split_dir, exist_ok=True)
        for file in split_files:
            shutil.copy(os.path.join(source_dir, file), os.path.join(split_dir, file))

In [ ]:
!ls "/content/drive/MyDrive/BINA PROJECT/indian Worker/indianWorker"
!find /content/drive -name "light"
!find /content/drive -name "severe"


ls: cannot access '/content/drive/MyDrive/BINA PROJECT/indian Worker/indianWorker': No such file or directory
/content/drive/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/split_data_cnn/train/light
/content/drive/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/split_data_cnn/val/light
/content/drive/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/split_data_cnn/test/light
/content/drive/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/light
/content/drive/.Encrypted/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/split_data_cnn/train/light
/content/drive/.Encrypted/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/split_data_cnn/val/light
/content/drive/.Encrypted/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/split_data_cnn/test/light
/content/drive/.Encrypted/MyDrive/Colab Notebooks/BINA PROJECT/indian Worker/indianWorker/light
/content/drive/MyDrive/Colab Notebooks/BINA PR

In [ ]:
import random
import shutil


output_dir = "/content/drive/MyDrive/BINA PROJECT/indian Worker/indianWorker/split_data_resnet50"

# חלוקת התמונות
split_data(source_dir_light, output_dir, "light", test_split, val_split)
split_data(source_dir_severe, output_dir, "severe", test_split, val_split)

In [ ]:
# --- טעינת הדאטה ---
train_data = tf.keras.utils.image_dataset_from_directory(
    os.path.join(output_dir, "train"),
    labels='inferred',
    label_mode='binary',  # סיווג בינארי
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True
)

train_data = train_data.unbatch()

# Oversampling לקטגוריה 'Severe'
def oversample_category(dataset, category_label):
    images, labels = [], []
    for x, y in dataset:
        images.append(x)
        labels.append(y)
        if tf.reduce_all(y == category_label):  # שכפול הקטגוריה החלשה
            images.append(x)
            labels.append(y)

    # בדיקה שכל הטנסורים הם באותו גודל
    assert all(img.shape == images[0].shape for img in images), "Shapes do not match!"

    # החזרת דאטהסט
    return tf.data.Dataset.from_tensor_slices((tf.stack(images), tf.stack(labels)))


Found 644 files belonging to 2 classes.


In [ ]:
# --- טעינת הדאטה ---
train_data = tf.keras.utils.image_dataset_from_directory(
    os.path.join(output_dir, "train"),
    labels='inferred',
    label_mode='binary',  # סיווג בינארי
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True
)

val_data = tf.keras.utils.image_dataset_from_directory(
    os.path.join(output_dir, "val"),
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    image_size=img_size
)

test_data = tf.keras.utils.image_dataset_from_directory(
    os.path.join(output_dir, "test"),
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    image_size=img_size
)


Found 644 files belonging to 2 classes.
Found 299 files belonging to 2 classes.
Found 99 files belonging to 2 classes.


In [ ]:
# פריסת קבוצות (Unbatch) כך שכל תמונה תעמוד בפני עצמה
train_data = train_data.unbatch()

# שינוי גודל של התמונות ל-224x224 כדי להבטיח אחידות
resize_layer = tf.keras.layers.Resizing(224, 224)
train_data = train_data.map(lambda x, y: (resize_layer(x), y))


In [ ]:
# נירמול הפיקסלים לטווח 0-1
normalization_layer = layers.Rescaling(1./255)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomBrightness(0.2),  # שינוי בבהירות
    layers.RandomContrast(0.2)    # שינוי בניגודיות
])


train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
test_data = test_data.map(lambda x, y: (normalization_layer(x), y))


NameError: name 'layers' is not defined

In [ ]:
# --- בניית המודל ---
# טעינת המודל המאומן מראש (EfficientNetB0)
base_model = tf.keras.applications.EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)


# הקפאת השכבות של המודל המאומן
base_model.trainable = False

# הוספת שכבות מותאמות אישית
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # סיווג בינארי: קל/חמור
])

# קומפילציה של המודל
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # קצב למידה מהיר יותר
    loss='binary_crossentropy',
    metrics=['accuracy']
)



In [ ]:
# --- הגדרת מספר האפוקים ---
epochs = 10  # מספר האפוקים לאימון הראשוני

# --- Early Stopping ---
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # אפשר גם 'val_accuracy' לפי מטרתך
    patience=5,  # מספר האפוקים ללא שיפור לפני עצירה
    restore_best_weights=True  # שחזור המשקלים הטובים ביותר
)

# --- משקלים לקטגוריות ---
class_weights = {0: 1.0, 1: 2.0}  # לדוגמה, עדיפות לקטגוריה 'Severe'

# --- אימון המודל הראשוני ---
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    class_weight=class_weights,  # הוספת משקלים לקטגוריות
    callbacks=[early_stopping]  # הוספת Early Stopping
)


In [ ]:
# --- שחרור שכבות נוספות ב-ResNet50 ---
for layer in base_model.layers:
    layer.trainable = True

# קומפילציה מחדש עם קצב למידה מהיר יותר
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # קצב למידה מהיר
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# --- Fine-Tuning ---
fine_tune_epochs = 10
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=fine_tune_epochs
)

In [ ]:
# --- ויזואליזציה לאחר Fine-Tuning ---
plt.figure(figsize=(8, 5))
plt.plot(history_fine.history['accuracy'], label='Training Accuracy (Fine-Tune)')
plt.plot(history_fine.history['val_accuracy'], label='Validation Accuracy (Fine-Tune)')
plt.title('Accuracy over Epochs (Fine-Tuning)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history_fine.history['loss'], label='Training Loss (Fine-Tune)')
plt.plot(history_fine.history['val_loss'], label='Validation Loss (Fine-Tune)')
plt.title('Loss over Epochs (Fine-Tuning)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# --- הערכת המודל על קבוצת הבדיקה ---
print("\nEvaluating on Test Data...")
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# חיזוי על קבוצת הבדיקה
y_true = []
y_pred = []
for images, labels in test_data:
    predictions = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((predictions > 0.5).astype(int).flatten())

# --- חישוב מדדים נוספים ---
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Light", "Severe"]))

In [ ]:
# מטריצת בלבול
conf_matrix = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# ויזואליזציה של מטריצת הבלבול
plt.figure(figsize=(6, 6))
plt.imshow(conf_matrix, cmap="Blues", interpolation='nearest')
plt.colorbar()
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.xticks([0, 1], ["Light", "Severe"])
plt.yticks([0, 1], ["Light", "Severe"])
plt.show()